### Karm Bandit - Course de chevaux
## Optimisez vos paris en misant sur le meilleur cheval !

Fabrice Mulotti 20/01/2022


![Alt text](static/course.jpg)

Imaginons une course de 15 chevaux, assez proches en performances.
Nous devons déterminer pour "The booky" notre meilleur stratégie pour nos paris.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class karm():
    __version__ = "1.2"
    def __init__(self,k,seed,epsilon, sigma ,initial_value):
        np.random.seed(seed)
        # nombre de bras
        self.k=k
        
        # e-greedy
        self.epsilon = epsilon

        # initialisation des moyennes des k bras
        self.mu      = np.random.normal(loc=100,scale=25,size=self.k)
        for i in range(self.k):
            if self.mu[i] <0 :
                self.mu[i]=0
        
        # ecart type pour chaque bras
        self.sigma   = np.full((self.k),sigma)

        self.initial_value = initial_value
        
        # 
        self.init_tableau()
    
    def version(self):
        #renvoie la version du code
        return self.__version__
    
    def my_argmax(self,my_array):
        # notre argmax
        my_max=-5.0
        my_list_of_max = []
        for i in range(0,my_array.shape[0]):
            if my_array[i] > my_max:
                my_max=my_array[i]
                my_list_of_max = [ i ]
            else:
                if my_array[i] == my_max:
                    my_list_of_max.append(i)
        return(np.random.choice(my_list_of_max))
          
    def init_tableau(self):
        # comptage du nombre de fois qu'un bras est actionné
        self.n=np.zeros((self.k))
        # init de la récompense par bras
        self.Reward=np.full((self.k),self.initial_value)
                    
    def show(self):
        print("k   = ",self.k)
        # print("mu  = ",self.mu)
        # print("sig = ",self.sigma)
        print("n   = ",self.n)
        print("R   = ",self.Reward)
     
    def max_reward(self):
        return(np.max(self.mu))
    
    def env(self,arm):
        # renvoie la récompense en fonction du bras actionné
        q=np.random.normal(self.mu[arm],self.sigma[arm])
        if (q < 0 ):
                q=0
        # self.Reward[arm]=(self.Reward[arm]*self.n[arm]+q)/(self.n[arm]+1)
        Reward_n = self.Reward[arm]
        self.n[arm]=self.n[arm]+1

        self.Reward[arm] = Reward_n + 1/self.n[arm] * (q - Reward_n)
        return(q)
    
    def action(self,t):
        arm=self.my_argmax(self.Reward)

        return(arm)
    
    def simulation(self,ntry,nsample):
        result=np.zeros((ntry,nsample))
        for i in range(ntry):
            self.init_tableau()
            for j in range(nsample):
                action=self.action(j)
                q=self.env(action)
                result[i,j]=q
        rmean = np.mean(result,axis=0)
        perte = self.max_reward() - rmean
        cumul_perte = np.cumsum(perte)
        cumul_reward = np.cumsum(rmean)
        return(rmean,cumul_perte)

In [ ]:
# Paramètres fixes !
k=15
seed=150
sigma = 100

# Paramètre que vous pouvez modifier !
epsilon = 0.01
initial_value =  250.0
dummy=karm(k,seed,epsilon, sigma, initial_value )
dummy.show()

![Alt text](static/frankie.jpg)

In [ ]:
recompense_moyenne , cumul_regret = dummy.simulation(400,400)
# rm_test,cumul_regret_test = dummy.simulation(1,20)
print("Cumul des récompenses : ",np.round(np.sum(recompense_moyenne),0),"$ vs initial 59847$")
print("Cumul des regret      : ",np.round(np.sum(cumul_regret[-1]),0),"$ vs initial 7082$")
dummy.show()

## Résultats 

In [ ]:
# A exécuter seulement à la première exécution pour garder une baseline
recompense_moyenne_initiale = recompense_moyenne
regret_initial = cumul_regret

In [ ]:
plt.figure()

plt.plot(recompense_moyenne_initiale,color='red')

plt.plot(recompense_moyenne,color='blue')

plt.show()

In [ ]:
plt.figure()
plt.plot(regret_initial,color = 'red')
plt.plot(cumul_regret,color='blue')


plt.show()